In [35]:
# Libraries needed 

import pandas as pd
import numpy as np
import requests
import io
import re
import nltk  
# download the required tokenizer
nltk.download('punkt') 
from nltk.tokenize import word_tokenize

# download the stopwords from NLTK
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# Create stemmer
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

# Create Lemmatizer
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


lemmatizer = WordNetLemmatizer()
# Downloading the csv file from your GitHub account

url = "https://raw.githubusercontent.com/20j01acs029/csc423-midterm/main/midterm/jumia%20file.csv" 
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe

jd = pd.read_csv(io.StringIO(download.decode('utf-8')))

#Data frame info 
jd.info()

# Printing out the first 5 rows of the dataframe
jd.head()



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   web-scraper-order      109 non-null    object
 1   web-scraper-start-url  109 non-null    object
 2   container_link         109 non-null    object
 3   container_link-href    109 non-null    object
 4   title                  109 non-null    object
 5   price                  108 non-null    object
 6   description            108 non-null    object
 7   features               74 non-null     object
 8   image-src              20 non-null     object
 9   ratings                108 non-null    object
 10  ratings-href           79 non-null     object
dtypes: object(11)
memory usage: 9.5+ KB


,web-scraper-order,web-scraper-start-url,container_link,container_link-href,title,price,description,features,image-src,ratings,ratings-href
0,1677256730-15,https://www.jumia.co.ke/computing/,Generic Type C Micro USB SD TF OTG Card Reader...,https://www.jumia.co.ke/generic-type-c-micro-u...,Generic Type C Micro USB SD TF OTG Card Reader...,KSh 390,+ shipping from KSh 92 to Embakasi East - Utaw...,NaN,NaN,(No ratings available),NaN
1,1677256742-16,https://www.jumia.co.ke/computing/,HP EliteBook 820 Core I5 4GB RAM 500gb Hdd Sli...,https://www.jumia.co.ke/hp-elitebook-820-core-...,HP EliteBook 820 Core I5 4GB RAM 500gb Hdd Sli...,"KSh 18,900",+ shipping from KSh 166 to Embakasi East - Uta...,Internal Storage 500GB,NaN,(No ratings available),NaN
2,1677256742-17,https://www.jumia.co.ke/computing/,HP EliteBook 820 Core I5 4GB RAM 500gb Hdd Sli...,https://www.jumia.co.ke/hp-elitebook-820-core-...,HP EliteBook 820 Core I5 4GB RAM 500gb Hdd Sli...,"KSh 18,900",+ shipping from KSh 166 to Embakasi East - Uta...,Graphics Integrated,NaN,(No ratings available),NaN
3,1677256742-18,https://www.jumia.co.ke/computing/,HP EliteBook 820 Core I5 4GB RAM 500gb Hdd Sli...,https://www.jumia.co.ke/hp-elitebook-820-core-...,HP EliteBook 820 Core I5 4GB RAM 500gb Hdd Sli...,"KSh 18,900",+ shipping from KSh 166 to Embakasi East - Uta...,1 USB 3.0,NaN,(No ratings available),NaN
4,1677256742-19,https://www.jumia.co.ke/computing/,HP EliteBook 820 Core I5 4GB RAM 500gb Hdd Sli...,https://www.jumia.co.ke/hp-elitebook-820-core-...,HP EliteBook 820 Core I5 4GB RAM 500gb Hdd Sli...,"KSh 18,900",+ shipping from KSh 166 to Embakasi East - Uta...,Intel® Ethernet Connection I219-V 10/100/1000 ...,NaN,(No ratings available),NaN


In [36]:
#Drop and rename columns
X= jd.drop(columns=['web-scraper-start-url','container_link-href','ratings-href', 'image-src','ratings'])
X.rename(columns={
        'web-scraper-order': 'N.o_order', 
        'container_link': 'Subject'
    }, inplace=True)
jd = X
jd[['Order_No','Unit']] = jd['N.o_order'].str.split('-',expand=True)

jd.drop('N.o_order', axis=1, inplace=True)

#Text processing

# Define a regular expression pattern to match URLs
url_pattern = re.compile(r'https?://\S+|www\.\S+')

# Remove URLs from all columns of the DataFrame
jd = jd.replace(to_replace=url_pattern, value='', regex=True)

# Define a regular expression pattern to match special characters and punctuations
special_char_pattern = re.compile(r'[^A-Za-z0-9 ]+')

# Remove special characters and punctuations from all columns of the DataFrame
jd = jd.replace(to_replace=special_char_pattern, value=' ', regex=True)

# Convert all text to lowercase
jd = jd.applymap(lambda s: s.lower() if type(s) == str else s)

#Checl null values
jd.isnull().sum()

Subject         0
title           0
price           1
description     1
features       35
Order_No        0
Unit            0
dtype: int64

In [37]:
# fill missing values in features with 'not avilable'
jd.fillna({'features': 'not avilable'}, inplace=True)

# drop rows with missing values
jd.dropna(inplace=True)


In [38]:
jd.isnull().sum()

Subject        0
title          0
price          0
description    0
features       0
Order_No       0
Unit           0
dtype: int64

In [39]:
jd.head()

,Subject,title,price,description,features,Order_No,Unit
0,generic type c micro usb sd tf otg card reader...,generic type c micro usb sd tf otg card reader...,ksh 390,shipping from ksh 92 to embakasi east utaw...,not avilable,1677256730,15
1,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,ksh 18 900,shipping from ksh 166 to embakasi east uta...,internal storage 500gb,1677256742,16
2,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,ksh 18 900,shipping from ksh 166 to embakasi east uta...,graphics integrated,1677256742,17
3,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,ksh 18 900,shipping from ksh 166 to embakasi east uta...,1 usb 3 0,1677256742,18
4,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,ksh 18 900,shipping from ksh 166 to embakasi east uta...,intel ethernet connection i219 v 10 100 1000 ...,1677256742,19


In [40]:

# Define a tokenizer function
def tokenize_text(text):
    tokens = word_tokenize(text)
    return ' '.join(tokens)
# Create a new column in the DataFrame to hold the tokenized text
jd['tokenized_text'] = ''

jd['tokenized_text'] = jd['description'].apply(tokenize_text)



In [41]:
jd.head()



,Subject,title,price,description,features,Order_No,Unit,tokenized_text
0,generic type c micro usb sd tf otg card reader...,generic type c micro usb sd tf otg card reader...,ksh 390,shipping from ksh 92 to embakasi east utaw...,not avilable,1677256730,15,shipping from ksh 92 to embakasi east utawala ...
1,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,ksh 18 900,shipping from ksh 166 to embakasi east uta...,internal storage 500gb,1677256742,16,shipping from ksh 166 to embakasi east utawala...
2,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,ksh 18 900,shipping from ksh 166 to embakasi east uta...,graphics integrated,1677256742,17,shipping from ksh 166 to embakasi east utawala...
3,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,ksh 18 900,shipping from ksh 166 to embakasi east uta...,1 usb 3 0,1677256742,18,shipping from ksh 166 to embakasi east utawala...
4,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,ksh 18 900,shipping from ksh 166 to embakasi east uta...,intel ethernet connection i219 v 10 100 1000 ...,1677256742,19,shipping from ksh 166 to embakasi east utawala...


In [42]:
# Apply the stopwords
#jd['tokenized_text'] = jd['tokenized_text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
# Define a function to remove stopwords from text
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return ' '.join(filtered_tokens)
# Apply the function to the DataFrame
jd['tokenized_text'] = jd['tokenized_text'].apply(remove_stopwords)

In [43]:
# Define a function to stem text
def stem_text(text):
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

# Apply the function to the DataFrame
jd['stemmed_text'] = jd['tokenized_text'].apply(stem_text)

# Display a sample of the stemmed data
jd['stemmed_text'].sample(3)

102    deliveri ksh 84 free deliveri order ksh 1 999 ...
107    deliveri ksh 84 free deliveri order ksh 1 999 ...
13        ship ksh 166 embakasi east utawala mihango gsu
Name: stemmed_text, dtype: object

In [44]:
jd.head()

,Subject,title,price,description,features,Order_No,Unit,tokenized_text,stemmed_text
0,generic type c micro usb sd tf otg card reader...,generic type c micro usb sd tf otg card reader...,ksh 390,shipping from ksh 92 to embakasi east utaw...,not avilable,1677256730,15,shipping ksh 92 embakasi east utawala mihango gsu,ship ksh 92 embakasi east utawala mihango gsu
1,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,ksh 18 900,shipping from ksh 166 to embakasi east uta...,internal storage 500gb,1677256742,16,shipping ksh 166 embakasi east utawala mihango...,ship ksh 166 embakasi east utawala mihango gsu
2,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,ksh 18 900,shipping from ksh 166 to embakasi east uta...,graphics integrated,1677256742,17,shipping ksh 166 embakasi east utawala mihango...,ship ksh 166 embakasi east utawala mihango gsu
3,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,ksh 18 900,shipping from ksh 166 to embakasi east uta...,1 usb 3 0,1677256742,18,shipping ksh 166 embakasi east utawala mihango...,ship ksh 166 embakasi east utawala mihango gsu
4,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,hp elitebook 820 core i5 4gb ram 500gb hdd sli...,ksh 18 900,shipping from ksh 166 to embakasi east uta...,intel ethernet connection i219 v 10 100 1000 ...,1677256742,19,shipping ksh 166 embakasi east utawala mihango...,ship ksh 166 embakasi east utawala mihango gsu


In [45]:
# Define a function to lemmatize text
def lemmatize_text(text):
    words = text.split()
    pos_tags = [get_wordnet_pos(word) for word in words]
    lemmatized_words = [lemmatizer.lemmatize(word, pos=pos) for word, pos in zip(words, pos_tags)]
    return ' '.join(lemmatized_words)

# Define a function to get WordNet part-of-speech tags
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# Apply the function to the DataFrame
jd['lemmatized_text'] = jd['stemmed_text'].apply(lemmatize_text)

# Display a sample of the lemmatized data
jd['lemmatized_text'].sample(10)

71        ship ksh 84 embakasi east utawala mihango gsu
90    deliveri ksh 102 free deliveri order ksh 1 999...
9        ship ksh 166 embakasi east utawala mihango gsu
92    deliveri ksh 102 free deliveri order ksh 1 999...
8        ship ksh 166 embakasi east utawala mihango gsu
6        ship ksh 166 embakasi east utawala mihango gsu
7        ship ksh 166 embakasi east utawala mihango gsu
12       ship ksh 166 embakasi east utawala mihango gsu
25       ship ksh 102 embakasi east utawala mihango gsu
66       ship ksh 166 embakasi east utawala mihango gsu
Name: lemmatized_text, dtype: object

In [46]:
# Save DataFrame to CSV file
jd.to_csv('new_jumia.csv', index=False)